In [ ]:
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
import pickle
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Dropout # new!
from keras.layers import BatchNormalization # new!
from keras import regularizers # new! 
from tensorflow.keras.optimizers import SGD
from keras.layers import Flatten, Conv2D, MaxPooling2D # new!
from keras.callbacks import ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data2 = 'drive/MyDrive/traffic-signs-data'
infile = open(data2+'/train.p','rb')
train = pickle.load(infile)
infile.close()
infile = open(data2+'/test.p','rb')
test = pickle.load(infile)
infile.close()
infile = open(data2+'/valid.p','rb')
valid = pickle.load(infile)
infile.close()

In [ ]:
def data_augment(image):
    rows= image.shape[0]
    cols = image.shape[1]
    
    # rotation
    M_rot = cv2.getRotationMatrix2D((cols/2,rows/2),10,1)
    
    # Translation
    M_trans = np.float32([[1,0,3],[0,1,6]])
    
    
    img = cv2.warpAffine(image,M_rot,(cols,rows))
    img = cv2.warpAffine(img,M_trans,(cols,rows))
    #img = cv2.warpAffine(img,M_aff,(cols,rows))
    
    # Bilateral filtering
    img = cv2.bilateralFilter(img,9,75,75)
    return img

In [ ]:
x_train = []
y_train = []
for i in range(len(train['features'])):
    image = train['features'][i]
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image = image/255
    image = image[:,:,np.newaxis]
    x_train.append(image)
    y_train.append(train['labels'][i])
x_train = np.array(x_train)
y_train = np.array(y_train)

x_test = []
y_test = []
for i in range(len(test['features'])):
    image = test['features'][i]
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image = image/255
    image = image[:,:,np.newaxis]
    x_test.append(image)
    y_test.append(test['labels'][i])
x_test = np.array(x_test)
y_test = np.array(y_test)

x_valid = []
y_valid = []
for i in range(len(valid['features'])):
    image = valid['features'][i]
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image = image/255
    image = image[:,:,np.newaxis]
    x_valid.append(image)
    y_valid.append(valid['labels'][i])
x_valid = np.array(x_valid)
y_valid = np.array(y_valid)

In [ ]:
n_classes = 43

In [ ]:
model_conv = keras.Sequential()
model_conv.add(Conv2D(32, kernel_size=(3, 3),activation='relu', input_shape=(32, 32, 1)))
model_conv.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model_conv.add(MaxPooling2D(pool_size=(2, 2)))
model_conv.add(BatchNormalization())
model_conv.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model_conv.add(MaxPooling2D(pool_size=(2, 2)))
model_conv.add(BatchNormalization())
model_conv.add(Dropout(0.25))
model_conv.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model_conv.add(MaxPooling2D(pool_size=(2, 2)))
model_conv.add(BatchNormalization())
model_conv.add(Dropout(0.5))
model_conv.add(Flatten())
model_conv.add(Dense(128, activation='relu'))
model_conv.add(Dropout(0.5))
model_conv.add(Dense(n_classes, activation='softmax'))

In [ ]:
model_conv.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
filepath="German_Traffic_ConvNetworkModel.hdf5"
checkpoint_conv = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list_conv = [checkpoint_conv]

In [ ]:
from numpy import array
from numpy import argmax
from tensorflow.keras.utils import to_categorical
# one hot encode
encoded1 = to_categorical(y_train)
print(encoded1.shape)
encoded2 = to_categorical(y_test)
encoded3 = to_categorical(y_valid)


(34799, 43)


In [ ]:
y_train = encoded1
y_test = encoded2
y_valid = encoded3

In [ ]:
y_train.shape

(34799, 43)

In [ ]:
#model_conv = tf.keras.models.load_model('drive/MyDrive/partly_trained.h5')

In [ ]:
model_conv.fit(x_train, y_train,callbacks=callbacks_list_conv, batch_size=128, epochs=70, verbose=1,validation_data=(x_valid, y_valid))

Epoch 1/70
272/272 [==============================] - 149s 544ms/step - loss: 2.3739 - accuracy: 0.3967 - val_loss: 4.7567 - val_accuracy: 0.0544
Epoch 2/70
272/272 [==============================] - 146s 538ms/step - loss: 0.6017 - accuracy: 0.8160 - val_loss: 2.5656 - val_accuracy: 0.3104
Epoch 3/70
272/272 [==============================] - 146s 536ms/step - loss: 0.2944 - accuracy: 0.9095 - val_loss: 0.1429 - val_accuracy: 0.9571
Epoch 4/70
272/272 [==============================] - 145s 535ms/step - loss: 0.1949 - accuracy: 0.9386 - val_loss: 0.0971 - val_accuracy: 0.9712
Epoch 5/70
272/272 [==============================] - 146s 535ms/step - loss: 0.1385 - accuracy: 0.9578 - val_loss: 0.1029 - val_accuracy: 0.9721
Epoch 6/70
272/272 [==============================] - 145s 534ms/step - loss: 0.1145 - accuracy: 0.9650 - val_loss: 0.0675 - val_accuracy: 0.9800
Epoch 7/70
272/272 [==============================] - 145s 533ms/step - loss: 0.0932 - accuracy: 0.9721 - val_loss: 0.0629 -

KeyboardInterrupt: ignored

In [ ]:
model_conv.evaluate(x_test,y_test)

395/395 [==============================] - 14s 36ms/step - loss: 0.0970 - accuracy: 0.9770


[0.09696297347545624, 0.9769596457481384]

In [ ]:
model_conv.save('drive/MyDrive/trained.h5')